In [3]:
# default_exp engineering.nbdev

%reload_ext autoreload
%autoreload 2

# 文档特征提取
文档特征提取的目的是用特征来表示文档。

常用的文档特征提取方法有:

## bag-of-words model
This model transforms each document to a fixed-length vector of integers. For example, given the sentences:

    John likes to watch movies. Mary likes movies too.

    John also likes to watch football games. Mary hates football.

The model outputs the vectors:

    [1, 2, 1, 1, 2, 1, 1, 0, 0, 0, 0]

    [1, 1, 1, 1, 0, 1, 0, 1, 2, 1, 1]

Bag-of-words models are surprisingly effective, but have several weaknesses.

首先，他们会丢失所有有关单词顺序的信息：“约翰喜欢玛丽”和“玛丽喜欢约翰”对应于相同的向量。 There is a solution: bag of n-grams models consider word phrases of length n to represent documents as fixed-length vectors to capture local word order but suffer from data sparsity and high dimensionality.

其次，该模型不会尝试学习基础单词的含义，因此，向量之间的距离并不总是反映出含义上的差异。 

## TF-IDF
TF-IDF是在词袋基础上的改进，对出现频率比较高的词进行降权(除以IDF)

TF-IDF（term frequency–inverse document frequency）是一种统计方法，用以评估一字词对于一个文件集或一个语料库中的其中一份文件的重要程度。

字词的重要性随着它在文件中出现的次数成正比增加，但同时会随着它在语料库中出现的频率成反比下降。

$$TFIDF=TF*IDF$$

### 词频 (term frequency, TF)

指的是某一个给定的词语在该文件中出现的次数。这个数字通常会被归一化（分子一般小于分母 区别于IDF），以防止它偏向长的文件。（同一个词语在长文件里可能会比短文件有更高的词频，而不管该词语重要与否。）
词频（TF）=某个词在文章中的出现次数
考虑到文章有长短之分，为了便于不同文章的比较，进行”词频”标准化。

$$TF=\frac{某个词在文章中的出现次数}{文章的总词数}$$
或者
$$TF=\frac{某个词在文章中的出现次数}{该训练文本中出现最多次的词数}$$

### 逆向文件频率 (inverse document frequency, IDF)

是一个词语普遍重要性的度量。某一特定词语的IDF，可以由总文件数目除以包含该词语之文件的数目，再将得到的商取对数得到。

$$IDF=log(\frac{总文档数}{包含该词的文档数+1})$$
如果一个词越常见，那么分母就越大，逆文档频率就越小越接近0。分母之所以要加1，是为了避免分母为0（即所有文档都不包含该词）。

## Embeding
一般用doc中所有word的embedding的均值，来表示这个doc的特征

# sklearn中用于文档特征化的类

CountVectorizer与TfidfVectorizer，这两个类都是特征数值计算的常见方法。对于每一个训练文本，CountVectorizer只考虑每种词汇在该训练文本中出现的频率，而TfidfVectorizer除了考量某一词汇在当前训练文本中出现的频率之外，同时关注包含这个词汇的其它训练文本数目的倒数。相比之下，训练文本的数量越多，TfidfVectorizer这种特征量化方式就更有优势。

## CountVectorizer
实现了词袋

Convert a collection of text documents to a matrix of token counts

This implementation produces a sparse representation of the counts using
`scipy.sparse.csr_matrix`.

### 默认参数

In [ ]:
CountVectorizer(
    input='content',
    encoding='utf-8',
    decode_error='strict',
    strip_accents=None,
    lowercase=True,
    preprocessor=None,
    tokenizer=None,
    stop_words=None,
    token_pattern='(?u)\\b\\w\\w+\\b',
    ngram_range=(1, 1),
    analyzer='word',
    max_df=1.0,
    min_df=1,
    max_features=None,
    vocabulary=None,
    binary=False,
    dtype=<class 'numpy.int64'>,
)

# \w: 匹配字母、数组、下划线。注意不会匹配到标点符号

In [5]:
from sklearn.feature_extraction.text import CountVectorizer


In [6]:
X_test = ['没有 你 的 地方 都是 他乡 没有 。 . , : ',
                  '没有 你 的 旅行 都是 流浪']

In [7]:
count_vec = CountVectorizer()

### count_vec.fit(raw_documents, y=None)
Learn a vocabulary dictionary of all tokens in the raw documents.

In [8]:
count_vec.fit(X_test)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

可以看到，默认情况下只有一个字母的 'I' 被过滤掉了！

In [9]:
count_vec.vocabulary_

{'没有': 3, '地方': 1, '都是': 5, '他乡': 0, '旅行': 2, '流浪': 4}

### count_vec.transform(X_test)
Transform documents to document-term matrix.

In [10]:
count_vec.transform(X_test)

<2x6 sparse matrix of type '<class 'numpy.int64'>'
	with 8 stored elements in Compressed Sparse Row format>

(index1,index2) count中：
* index1表示为第几个句子或者文档，
* index2为所有语料库中的单词组成的词典的序号。
* count为在这个文档中这个单词出现的次数。
* 注意：这样统计时丢失了word在text中的位置信息!!!

In [11]:
print(count_vec.transform(X_test))

  (0, 0)	1
  (0, 1)	1
  (0, 3)	2
  (0, 5)	1
  (1, 2)	1
  (1, 3)	1
  (1, 4)	1
  (1, 5)	1


In [ ]:
doc-token矩阵：
每一行表示一个文档，每一列表示相应编号的token。值为token在doc中出现的频数。
这一步已经将doc转化成了由词频表示的特征

In [15]:
count_vec.transform(X_test).toarray()

array([[1, 1, 0, 2, 0, 1],
       [0, 0, 1, 1, 1, 1]])

## TfidfVectorizer

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

TfidfVectorizer(
    input='content',
    encoding='utf-8',
    decode_error='strict',
    strip_accents=None,
    lowercase=True,
    preprocessor=None,
    tokenizer=None,
    analyzer='word',
    stop_words=None,
    token_pattern='(?u)\\b\\w\\w+\\b',
    ngram_range=(1, 1),
    max_df=1.0,
    min_df=1,
    max_features=None,
    vocabulary=None,
    binary=False,
    dtype=<class 'numpy.float64'>,
    norm='l2',
    use_idf=True,
    smooth_idf=True,
    sublinear_tf=False,
)

* norm : 'l1', 'l2' or None, optional (default='l2')  
    把输出向量的长度归一化  
    Each output row will have unit norm, either:  
    * 'l2': Sum of squares of vector elements is 1. The cosine  
    similarity between two vectors is their dot product when l2 norm has been applied.
    * 'l1': Sum of absolute values of vector elements is 1.  
    See :func:`preprocessing.normalize`
* token_pattern='(?u)\\b\\w\\w+\\b'
注意默认值会过滤掉单个汉字

In [16]:
tfidf = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b")

In [17]:
tfidf.fit(X_test)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

### 获取vocab

In [18]:
tfidf.vocabulary_

{'没有': 4, '你': 1, '的': 6, '地方': 2, '都是': 7, '他乡': 0, '旅行': 3, '流浪': 5}

In [19]:
tfidf.stop_words_

set()

In [20]:
tfidf.get_feature_names()

['他乡', '你', '地方', '旅行', '没有', '流浪', '的', '都是']

In [21]:
tfidf.transform(['a b 你 他乡']).toarray()

array([[0.81480247, 0.57973867, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ]])

### 一个细节问题

In [ ]:
\w匹配的是字母数字、下划线

In [ ]:
可以看到 分词以后把 '没.有'分为一个词，但是token_pattern=r"(?u)\b\w+\b" 没有'.', 实际的处理是把'没' '有'作为两个词来处理

In [1]:
X_test = ['没.有 你 的 地方 都是 他乡 没有 。 . , : ',
                  '没有 你 的 旅行 都是 流浪']

In [4]:
tfidf = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b")

In [5]:
tfidf.fit(X_test)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [6]:
tfidf.vocabulary_

{'没': 5,
 '有': 4,
 '你': 1,
 '的': 8,
 '地方': 2,
 '都是': 9,
 '他乡': 0,
 '没有': 6,
 '旅行': 3,
 '流浪': 7}

## 使用Gensim进行TfIdf

In [4]:
from gensim.models import TfidfModel
from gensim import corpora
from collections import defaultdict
"""

"""
raw_corpus = ["Human machine interface for lab abc computer applications",
              "A survey of user opinion of computer system response time",
              "The EPS user interface management system",
              "System and human system engineering testing of EPS",
              "Relation of user perceived response time to error measurement",
              "The generation of random binary unordered trees",
              "The intersection graph of paths in trees",
              "Graph minors IV Widths of trees and well quasi ordering",
              "Graph minors A survey"]
# Create a set of frequent words
stoplist = set('for a of the and to in'.split(' '))
# Lowercase each document, split it by white space and filter out stopwords
texts = [[word for word in document.lower().split() if word not in stoplist]
         for document in raw_corpus]
# Count word frequencies
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

# Only keep words that appear more than once
processed_corpus = [[token for token in text if frequency[token] > 1] for text in texts]
print(processed_corpus)
dictionary = corpora.Dictionary(processed_corpus)
print(dictionary)  # Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...)
print(dictionary.token2id)
"""
{'computer': 0, 'human': 1, 'interface': 2, 'response': 3, 'survey': 4, 'system': 5, 'time': 6, 'user': 7, 'eps': 8, 'trees': 9, 'graph': 10, 'minors': 11}
"""
new_doc = "Human computer interaction"
new_vec = dictionary.doc2bow(new_doc.lower().split())
print(new_vec)#[(0, 1), (1, 1)]
"""
The first entry in each tuple corresponds to the ID of the token in the dictionary, the second corresponds to the count of this token.
Note that "interaction" did not occur in the original corpus and so it was not included in the vectorization. 
"""
bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus]
print(bow_corpus)
# train the model
tfidf = TfidfModel(bow_corpus)
# transform the "system minors" string
print(tfidf[dictionary.doc2bow("system minors".lower().split())])#[(5, 0.5898341626740045), (11, 0.8075244024440723)]


[['human', 'interface', 'computer'], ['survey', 'user', 'computer', 'system', 'response', 'time'], ['eps', 'user', 'interface', 'system'], ['system', 'human', 'system', 'eps'], ['user', 'response', 'time'], ['trees'], ['graph', 'trees'], ['graph', 'minors', 'trees'], ['graph', 'minors', 'survey']]
Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...)
{'computer': 0, 'human': 1, 'interface': 2, 'response': 3, 'survey': 4, 'system': 5, 'time': 6, 'user': 7, 'eps': 8, 'trees': 9, 'graph': 10, 'minors': 11}
[(0, 1), (1, 1)]
[[(0, 1), (1, 1), (2, 1)], [(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)], [(2, 1), (5, 1), (7, 1), (8, 1)], [(1, 1), (5, 2), (8, 1)], [(3, 1), (6, 1), (7, 1)], [(9, 1)], [(9, 1), (10, 1)], [(9, 1), (10, 1), (11, 1)], [(4, 1), (10, 1), (11, 1)]]
[(5, 0.5898341626740045), (11, 0.8075244024440723)]


# nb_export

In [4]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted engineering_nbdev.ipynb.
Converted index.ipynb.


In [7]:
!nbdev_build_docs

No notebooks were modified
converting /Users/luoyonggui/PycharmProjects/nbdevlib/index.ipynb to README.md
